In [1]:
import json
import pandas as pd
import sqlalchemy
import urllib
from sqlalchemy import create_engine
from sqlalchemy import Table, Column, Integer, String, MetaData, ForeignKey
from sqlalchemy import inspect

In [6]:
#Connect to MSSQL
params = urllib.parse.quote_plus("DRIVER={ODBC Driver 17 for SQL Server};SERVER=DESKTOP-R3NS070; DATABASE=Beer; UID=tlt23; PWD=password; Trusted_Connection=yes;")

#Engine
cnxn = create_engine("mssql+pyodbc:///?odbc_connect=%s" % params)

# If issues connecting in the future due to ODBC Driver, make sure that MS SQL Server Mgnt Studio is updated.


In [3]:
#opening JSON file and loading data
#orient='index' added to make sure data loaded long instead of wide
with open(r'D:\Datasets\Resume Projects\homebrew beer recipes\recipes_full.json.txt', encoding='utf-8-sig') as beer:
    beer = pd.read_json(beer, orient='index')

In [4]:
#to verify it loaded correctly
print(beer["name"].head(5))
print(beer["name"].tail(5))

0                           Vanilla Cream Ale
1          Avg. Perfect Northeast IPA (NEIPA)
2                Sierra Nevada Pale Ale Clone
3               Zombie Dust Clone - ALL GRAIN
4    Russian River Pliny the Elder (original)
Name: name, dtype: object
179455           Awesome Recipe
179456               Lime split
179457           Awesome Recipe
179458    Goldzacca Session IPA
179459           Awesome Recipe
Name: name, dtype: object


In [11]:
#Creading an ID column for the table
numbers = range(0,179455)
id = []
for number in numbers:
    id.append(number)


#Create SQL Table with Parameters and execute
sql = "CREATE TABLE BeerInfo(id INTEGER PRIMARY KEY, name nvarchar(MAX), method nvarchar(MAX), style nvarchar(MAX), batch float, og float, fg float, abv float, ibu float, [ph mash] real)"
cnxn.execute(sql)

#Create Dictionary of names w/ data 
beer1 = {"id":id,"name": beer["name"], "method":beer["method"], "style":beer["style"], "batch":beer["batch"], "og":beer["og"], "fg":beer["fg"], "abv":beer["abv"], "ibu":beer["ibu"], "ph mash":beer["ph mash"]}

#Check that Dict works
beer1["og"][0]

#Create datafrom from dictionary and Send to SQL
beer2 = pd.DataFrame.from_dict(beer1)
beer2.to_sql('BeerInfo', con=cnxn, index=False, if_exists='replace', method='multi', chunksize=100)

In [12]:
#loop through the outer most list and get the index of each element using enumerate. 
#Then, loop through each inner list, prepend the index to each of the 5 element lists and combine them into one main list, 
#data_id, and finally convert the main list into a dataframe that can be written to the database
## Example of list of lists w/ Fermentabels and why this is necessary to access to each element of the list
### Needed 'enumerate' in order to assign each element to the corresponding Beer
print(beer["fermentables"][0])

[[2.3810000000000002, 'American - Pale 2-Row', 37.0, 1.8, 44.7], [0.907, 'American - White Wheat', 40.0, 2.8, 17.0], [0.907, 'American - Pale 6-Row', 35.0, 1.8, 17.0], [0.227, 'Flaked Corn', 40.0, 0.5, 4.3], [0.227, 'American - Caramel / Crystal 20L', 35.0, 20.0, 4.3], [0.227, 'American - Carapils (Dextrine Malt)', 33.0, 1.8, 4.3], [0.113, 'Flaked Barley', 32.0, 2.2, 2.1], [0.34, 'Honey', 42.0, 2.0, 6.4]]


In [18]:
######################### New Table for Fermentables #########################

In [40]:
data_id = []
for ind, d in enumerate(beer["fermentables"]):
    for a in d:
        a.insert(0, ind)
        data_id.append(a)
df = pd.DataFrame(data_id)


fermnum = range(0,675287)
fermid = []
for number in fermnum:
    fermid.append(number)

In [15]:
df.columns = ['Beerid', 'Weight', 'GrainName', 'PPG', 'DegLitner', 'GrainBill']

In [16]:
sql = "CREATE TABLE Fermentables(ID INTEGER PRIMARY KEY, BeerID int NOT NULL, Weight float, GrainName nvarchar(MAX), PPG float, DegLitner float, GrainBill float)"
cnxn.execute(sql)

In [17]:
ferm1 = {"ID":fermid, "BeerID":df["Beerid"],"Weight": df["Weight"], "GrainName":df["GrainName"], "PPG":df["PPG"], "DegLitner":df["DegLitner"], "GrainBill":df["GrainBill"]}
ferm2 = pd.DataFrame.from_dict(ferm1)

ferm2.to_sql('Fermentables', con=cnxn, index=False, if_exists='replace', method='multi', chunksize=100)

In [19]:
######################### New Table for Hops #########################

In [20]:
hops_id = []
for ind, d in enumerate(beer["hops"]):
    for a in d:
        a.insert(0, ind)
        hops_id.append(a)
hops = pd.DataFrame(hops_id)

hopsnum = range(0,621055)
hopsid = []
for number in hopsnum:
    hopsid.append(number)

In [21]:
hops.columns = ['Beerid', 'Weight', 'HopsName', 'Type', 'AlphaAcids', 'Process', 'ProcessTime', 'IBU', "HopsBill"]

In [23]:
#ProcessTime is a column of character that needs to be split.
print(hops["ProcessTime"].head(5))

0    60 min
1    20 min
2     5 min
3    10 min
4    10 min
Name: ProcessTime, dtype: object


In [24]:
#split into numbers from strings
Process = hops["ProcessTime"].str.split(" ", n=1, expand=True)
Process = pd.DataFrame(Process)

In [25]:
#Sucessfully split
print(Process[0].head(5))
print(Process[1].head(5))

0    60
1    20
2     5
3    10
4    10
Name: 0, dtype: object
0    min
1    min
2    min
3    min
4    min
Name: 1, dtype: object


In [26]:
# Make sure all of Process 0 is numeric
Process[0] = pd.to_numeric(Process[0])


ValueError: Unable to parse string "Day" at position 4028

In [28]:
#"ValueError: Unable to parse string "Day" at position 4028"
##Meaning there was some scraped data that got mixed up between time and time unit
## Example:
    ## Should be - 
Process.loc[[4027]]


,0,1
4027,10,min


In [29]:
    ## Instead, it looks like:
Process.loc[[4028]]
    ##Intuitiviely, it should be 5 days, so correcting this

,0,1
4028,Day,5


In [30]:
## Count the number of "Day"'s in wrong column and make a list of the index numbers
x = Process.index[Process[0] == "Day"]
print(len(x))

2114


In [31]:
#There are 2,114 rows that incorrectly have "Day" However, we know the correct number is in Process[1]
#So we can just replace 'Day' with the correct number by referencing Process[1] and using our x list of index numbers
for indexnumber in x:
    Process[0][indexnumber] = Process[1][indexnumber]

In [32]:
# Use the same index numbers from list x to replace the numbers with 'days'
for indexnumber in x:
    Process.loc[indexnumber,1] = 'days'

In [33]:
# Make sure all of Process[ 0 ]is finally numeric
Process[0] = pd.to_numeric(Process[0])

In [34]:
#When investigating previous error, I found that there were a few '10080 days'
## But I know that 10080 minutes is how many minutes in a week
## So to correct this - 
z = Process.index[Process[0] == 10080]

for indexnumber in z:
    Process.loc[indexnumber, 0] = 7

In [35]:
#Same as for Process, there are some Weight that include units and forces the dtype to be character.
pd.to_numeric(hops["Weight"]) #The error code here tells you where the perps start and gives you an idea of how to correct it

ValueError: Unable to parse string "23 ml" at position 20729

In [38]:
#Slightly more straightforward approach than for Process
hops["Weight"] = hops["Weight"].astype(str) #convert column to strings so that the random ml's can be removed
hops["Weight"] = hops["Weight"].map(lambda x: x.rstrip(' ml')) #strip the extra
pd.to_numeric(hops["Weight"]) #Successfully transformed to numeric for SQL use.
print(hops["Weight"][20729])

23


In [39]:
#Create sql table Hops and execute
sql = "CREATE TABLE Hops(ID INTEGER PRIMARY KEY, BeerID int NOT NULL, Weight float, HopsName nvarchar(MAX), Type nvarchar(MAX), AlphaAcids float, Process nvarchar(MAX), ProcessTime float, ProcessUnit nvarchar(MAX), IBU float, HopsBill float)"
cnxn.execute(sql)

#Hops columns as Dictionary
hops1 = {"ID":hopsid, "BeerID":hops["Beerid"],"Weight": hops["Weight"], "HopsName":hops["HopsName"], "Type":hops["Type"], "AlphaAcids":hops["AlphaAcids"], "Process":hops["Process"], "ProcessTime":Process[0], "ProcessUnit":Process[1], "IBU":hops["IBU"], "HopsBill":hops["HopsBill"]}
hops2 = pd.DataFrame.from_dict(hops1)

#Add data to columns in SQL
hops2.to_sql('Hops', con=cnxn, index=False, if_exists='replace', method='multi', chunksize=100)